<a href="https://colab.research.google.com/github/maxigaarp/Gestion-de-datos-2022/blob/main/Proyecto_GDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Proyecto Grupal 

## Entrega: Domingo 29 de Agosto 2021

Grupos de 2 o 3 personas

Porcentaje: 33.3%

Este trabajo consistirá en armar una base de datos en SQL y un informe descriptivo de los datos y la investigación. El objetivo del proyecto es comparar la asistencia a clases en base a los datos meteorologicos de precipitación.

La base de datos debe tener la informacion de 3 diferentes fuentes de información:

 * Asistencia Declarada Mensual ([MINEDUC](http://datos.mineduc.cl/dashboards/19844/asistencia-declarada-mensual-por-ano/)) años: (2015-2019)
  Acá se incluyen los porcentajes de asistencia y la cantidad de dias trabajados de todos los alumnos en los distintos colegios de Chile 
 * Reportes de precipitacion Mensual ([CR2](https://www.cr2.cl/camels-cl/)): Datos historicos de precipitacion las distintas estaciones meteorologicas en Chile con localizacion geografica.
 * Localizacion Colegios (Propia/Google Maps): Datos de la ubicacion geográfica de los colegios en Chile.

Para esto generar esta base de datos se espera que se incluyan a lo menos 3 diferentes tablas:
 * alumnos: Informacion relevante de los alumnos. A lo menos incluir 3 atributos.
 * colegios: Informacion relevante de los colegios. A lo menos incluir 3 atributos, ademas:  nombre (o ID) de la estacion meteorologica más cercana y la distancia a esta, asumiendo latitud y longitud como coordenadas cartesianas.
 * estudia_en: Informacion relevante del vinculo entre el colegio, mes y alumnos. En la tabla se debe incorporar la informacion mensual para cada alumno en terminos de asistencia. 
 * Precipitación: información historica de precipitaciones por año, mes y estación meteorológica.

 
Cabe destacar que las bases de datos no se encuentran libres de errores por lo que para consolidar la informacion hay que solucionar problemas de consistencia, completitud, validez. Entre otros se busca que:

 * Que los datos para cada uno atributos se encuentren en el formato adecuado y unico. Para lograr esto revisar los atributos más importantes: asistencia y precipitacion.
 * Una unica forma de expresar valores nulos (unificar distintos tipos de descripcion de valores nulos, ejemplo: " ", NA, 0, -9999, etc). 


Cabe destacar que las tablas deben unificarse en SQL cargando las tablas mediante algún driver de R para DBMS (Rpostgres, RSQLite, etc...). Esto es con el objetivo de poder cargar el volumen de datos necesarios para el proyecto.


La evaluacion del proyecto consistirá en las siguientes partes:

* (2 pts) Documentacion de la base de datos, debe incluir: 
 - (0.5 pts) Diagrama Entidad-Relacion
 - (0.5 pts) Esquema de creacion de tabla SQL
 - (0.5 pts) Justificacion de su diseño
 - (0.5 pts) Resumen de lo desarrollado en el proyecto (como se generaron las variables, como se procesaron y que resultados se obtuvieron)

* (2 pt) Poblar la base de datos, para ello tomaremos en cuenta principalmente los 3 partes fundamentales en su creacion:
 - (0.5 pts) Seleccionar solo los pares (MRUN, RBD, AGNO) que tengan exactamente 10 meses distintos y cuyo promedio de asistencia sea mayor al 75% anual (puede calcular este promedio tomando directamente el promedio de ASIS_PROMEDIO)
 - (0.5 pts) Para los valores de precipitacion, seleccionar las estaciones meteorológicas con menos de 5 valores nulos en el intervalo de tiempo 2015-2019. Luego de esto, asumir que si hay un dato nulo para una estacion meteorológica, la precipitacion de ese mes fue 0, el promedio o una interpolacion lineal de los datos cercanos.
 - (1 pt) Para resolver algunas preguntas del proyecto y poblar correctamente la base de datos se necesitará calcular la distancia entre una estacion meteorológica y un colegio. En este paso es valido medir usando la distancia euclideana ($\sqrt{(x_1-x_2)^2+ (y_1-y_2)^2}$) entre ambas coordenadas geograficas y deberá tomar como validos solo los pares colegio-estacion a menos de 0.2 (paralelo-meridiano)


* (3.5 pts )Responder las siguientes preguntas:

  - (1 pt) Cuál es la comuna con menor y mayor precipitación mensual promedio (últimos 5 años)
  - (1 pt) Calcule la asistencia promedio por colegio para los distintos meses (promediando los valores de los últimos 5 años). Grafique la asistencia promedio para un colegios de: Arica, Santiago y otro de Valdivia.
  - (1 pt) Realice un gráfico comparativo (de puntos) entre asistencia promedio mensual (eje x) y la precipitacion (eje y) para las 3 siguientes comunas:
   - Arica
   - Santiago
   - Valdivia
  - (0.5 pts)Concluya el analisis describiendo las posibles explicaciones de su resultado y las formas para generalizar este estudio.

El codigo donde se genera la base de datos y el desarrollo del algoritmo de prediccion puede ser Colab o script de R, debe tener comentarios con el objetivo de guiar al ayudante en la corrección.

Para una mejor calidad en el informe puede usar graficos y tablas.

Datos disponibles en: https://drive.google.com/drive/folders/17Or8k6rhYvkaeEn_pD10za8A9jluYfUF?usp=sharing

#Descargar datos

##Asistencia

In [ ]:
library(tidyverse)

In [ ]:
library(stringr)

In [ ]:
system("gdown --id 1-q1ydcu6afA3LQ9uxlh9J9B9kvEJvrRs")

In [ ]:
unzip("/content/DatosProyecto.zip")

In [ ]:
lista <- list.files("/content/content/drive/MyDrive/Gestion de Datos/Datos Proyecto/Datos asistencia")
directorio <- "/content/content/drive/MyDrive/Gestion de Datos/Datos Proyecto/Datos asistencia/"

for (name in lista){
  if (str_detect(name, ".rar")){
    foldername<-substr(name, 1, nchar(name)-4)
    system(paste("mkdir 'Asistencia/",foldername,"'",sep=""))
    system(paste("unrar x '",directorio,name,"' 'Asistencia/",substr(name, 1, nchar(name)-4),"/'", sep=""),intern = T)
  }
  else {
    unzip(paste(directorio,name,sep=""), exdir="Asistencia")
  }
}


In [ ]:
lista_de_csvs <- list.files("Asistencia",pattern = ".(CSV|csv)$", recursive = TRUE)


## Precipitacion

In [ ]:
unzip("/content/content/drive/MyDrive/Gestion de Datos/Datos Proyecto/cr2_prAmon_2019.zip")

## Geolocalización




In [ ]:
geocoles <- read.csv("/content/content/drive/MyDrive/Gestion de Datos/Datos Proyecto/colesgeo.csv", row.names=1)

#Preguntas:

## (1 pt) Cuál es la comuna con menor y mayor precipitación mensual promedio (últimos 5 años)


  - (1 pt) Calcule la asistencia promedio por colegio para los distintos meses (promediando los valores de los últimos 5 años). Grafique la asistencia promedio para un colegios de: Arica, Santiago y otro de Valdivia.
.

## (1 pt) Realice un gráfico comparativo (de puntos) entre asistencia promedio mensual (eje x) y la precipitacion (eje y) para las 3 siguientes comunas:


   - Arica


   - Santiago


   - Valdivia


##(0.5 pts) Concluya el analisis describiendo las posibles explicaciones de su resultado y las formas para generalizar este estudio.